<a href="https://colab.research.google.com/github/Yuto-T-440/25s-MatDataSci-Tohoku/blob/main/CNN_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchaudio
!pip install torch-geometric
!pip install rdkit-pypi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
import numpy as np
from torch_geometric.data import Data
from torch_geometric.nn import radius_graph
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch.optim import Adam
import torch
import torch.nn.functional as F
from torch.nn import Linear, Module

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the .npz file
data = np.load('/content/drive/MyDrive/InformationTechnologyFundamental/DFT_all.npz',allow_pickle=True)

In [ ]:
print(list(data.keys()))

['compounds', 'atoms', 'freqs', 'vibmodes', 'zpves', 'U0', 'U298', 'H', 'S', 'G', 'Cv', 'Cp', 'coordinates', 'Vesp', 'Qmulliken', 'dipole', 'quadrupole', 'octupole', 'hexadecapole', 'rots', 'gap', 'Eee', 'Exc', 'Edisp', 'Etot', 'Eatomization']


In [ ]:
Zs = data['atoms']             # atomic numbers (per molecule)
Rs = data['coordinates']       # 3D positions (per molecule)

# Extract dipole components
dipoles = data['dipole']
dipole_x = np.array([d[0] for d in dipoles])
dipole_y = np.array([d[1] for d in dipoles])
dipole_z = np.array([d[2] for d in dipoles])

# Define targets
targets = ['U0', 'gap', 'H']
Y_all = np.column_stack([
    data['U0'], data['gap'], data['H'],
    dipole_x, dipole_y, dipole_z
])
Ys = list(Y_all)

In [ ]:
def build_graph(Z, R, y):
    x = torch.tensor(Z, dtype=torch.float).reshape(-1, 1)
    pos = torch.tensor(R, dtype=torch.float)

    y = torch.tensor(y, dtype=torch.float).reshape(1, -1)  # ✅ fix: ensures [1, 6] shape

    edge_index = radius_graph(pos, r=5.0, loop=False)
    return Data(x=x, pos=pos, edge_index=edge_index, y=y)

In [ ]:
print(torch.__version__)

2.6.0+cu124


In [ ]:
# PyTorch Geometric & 依存ライブラリのインストール（torch-cluster含む）
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric \
  -f https://data.pyg.org/whl/torch-2.6.0+cu124.html

Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.3 MB/s eta 0:00:00


In [ ]:
from torch_cluster import radius_graph, knn_graph

In [ ]:
dataset = [build_graph(z, r, y) for z, r, y in zip(Zs, Rs, Ys)]
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
class GNNModel(Module):
    def __init__(self, hidden_dim=64, output_dim=6):
        super().__init__()
        self.conv1 = GCNConv(1, hidden_dim)  # input: atomic number
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.lin1 = Linear(hidden_dim, hidden_dim)
        self.lin2 = Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)       # Global graph representation
        x = F.relu(self.lin1(x))
        out = self.lin2(x)                   # Multi-target output
        return out

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNNModel().to(device)
optimizer = Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.MSELoss()

In [ ]:
def get(self, idx):
    Z = self.Zs[idx]
    R = self.Rs[idx]
    y = self.Ys[idx]

    x = torch.tensor(Z, dtype=torch.float).reshape(-1, 1)
    pos = torch.tensor(R, dtype=torch.float)
    y = torch.tensor(y, dtype=torch.float)  # no reshape

    edge_index = radius_graph(pos, r=self.radius, loop=False)
    return Data(x=x, pos=pos, edge_index=edge_index, y=y)

In [ ]:
# Send model to train mode
model.train()

for epoch in range(1, 51):
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)

        # Forward pass
        pred = model(batch)  # shape [batch_size, 6]
        target = batch.y     # shape [batch_size, 6]

        # Confirm shapes match
        if pred.shape != target.shape:
            print(f"Shape mismatch: pred={pred.shape}, target={target.shape}")
            continue  # skip batch or raise an error

        # Loss and backprop
        loss = loss_fn(pred, target)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item() * batch.num_graphs

    avg_loss = total_loss / len(loader.dataset)
    print(f"Epoch {epoch:02d} | Loss: {avg_loss:.4f}")

Epoch 01 | Loss: 127264.6604
Epoch 02 | Loss: 120918.0735
Epoch 03 | Loss: 120821.6446
Epoch 04 | Loss: 120781.6703
Epoch 05 | Loss: 120609.2717
Epoch 06 | Loss: 120543.3042
Epoch 07 | Loss: 120460.6021
Epoch 08 | Loss: 120497.3469
Epoch 09 | Loss: 120416.7025
Epoch 10 | Loss: 119918.7116
Epoch 11 | Loss: 119139.3989
Epoch 12 | Loss: 118287.8450
Epoch 13 | Loss: 117135.0370
Epoch 14 | Loss: 112405.7080
Epoch 15 | Loss: 105412.4367
Epoch 16 | Loss: 102506.4348
Epoch 17 | Loss: 100153.2121
Epoch 18 | Loss: 98035.5094
Epoch 19 | Loss: 96548.6685
Epoch 20 | Loss: 95471.6698
Epoch 21 | Loss: 94573.1208
Epoch 22 | Loss: 94047.3128
Epoch 23 | Loss: 93500.1509
Epoch 24 | Loss: 93153.6383
Epoch 25 | Loss: 92981.2232
Epoch 26 | Loss: 92737.1286
Epoch 27 | Loss: 92448.0950
Epoch 28 | Loss: 92199.6846
Epoch 29 | Loss: 91990.8146
Epoch 30 | Loss: 91832.3412
Epoch 31 | Loss: 91546.4034
Epoch 32 | Loss: 91238.2474
Epoch 33 | Loss: 91056.4688
Epoch 34 | Loss: 90916.2208
Epoch 35 | Loss: 90723.8773
Epo